# Update Stock Historical Price/Volume Database

## 資料檔案設定

In [1]:
data_filepath = 'data/data.feather'  # .feather 儲存格式

## 程式執行

### 套件設定

In [2]:
import os
import pandas as pd
import yfinance as yf
from getStockList import get_stock_symbols
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterSession, LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
import datetime
import pytz

In [3]:
# Rate limiter and cached
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

# start a cached session
session = CachedLimiterSession(
    limiter=Limiter(RequestRate(10, Duration.SECOND*2)),
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("requests.cache"),
)

### 載入資料

In [4]:
# Get stock list
stock_symbol_df = get_stock_symbols()

stock_symbol_df  # Inspect stock symbol df

,symbol,name,market,industry,yf_symbol
0,1101,台泥,上市,水泥工業,1101.TW
1,1102,亞泥,上市,水泥工業,1102.TW
2,1103,嘉泥,上市,水泥工業,1103.TW
3,1104,環泥,上市,水泥工業,1104.TW
4,1108,幸福,上市,水泥工業,1108.TW
...,...,...,...,...,...
1796,9949,琉園,上櫃,文化創意業,9949.TWO
1797,9950,萬國通,上櫃,塑膠工業,9950.TWO
1798,9951,皇田,上櫃,電機機械,9951.TWO
1799,9960,邁達康,上櫃,運動休閒,9960.TWO


In [12]:
# LOAD DATA ##################
directory, filename = os.path.split(data_filepath)

# Check if the directory exists
if not os.path.exists(directory):
    # If not, create the directory
    os.makedirs(directory)
    print(f"Directory '{directory}' created.")

# load data
if os.path.exists(data_filepath):
    data_df = pd.read_feather(data_filepath)
    
else:  # Data file does not exist, download it from Yahoo Finance
    print('No current data file found, start to download')
    
    # Get stock symbols
    symbols = stock_symbol_df['yf_symbol'].to_list()

    # Download data
    data_df = yf.download(symbols, session=session, period="10y", progress=True)


# Sort date in acending order
data_df.sort_index(ascending=True, inplace=True)

### 清理資料

In [14]:
from tqdm.notebook import tqdm
import numpy as np

# Extract all the stock symbols from the MultiIndex
stock_symbols = data_df.columns.get_level_values(1)
stock_symbols = set(stock_symbols.to_list())

# Get the last row of data_df
last_row = data_df.iloc[-1]

# Remove symbol that do not have meaningful content
symbols_to_remove = []

for symbol in tqdm(stock_symbols):
    check_list = []
    check_list.append(last_row['Open'][symbol])
    check_list.append(last_row['High'][symbol])
    check_list.append(last_row['Low'][symbol])
    check_list.append(last_row['Close'][symbol])
    check_list.append(last_row['Adj Close'][symbol])
    check_list.append(last_row['Volume'][symbol])

    if np.any(np.isnan(check_list)):
        symbols_to_remove.append(symbol)

# Create a boolean mask for columns that should not be removed
mask = ~data_df.columns.get_level_values(1).isin(symbols_to_remove)

# Use the mask to filter the columns
data_df = data_df.loc[:, mask]

  0%|          | 0/1801 [00:00<?, ?it/s]

symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol
symbol


### 更新資料

In [17]:
# Get date information
now = datetime.datetime.now(tz=pytz.timezone('Asia/Taipei'))
date_today = now.date()
last_day_data = data_df.index[-1].date()

In [18]:
date_today + datetime.timedelta(days=1)

datetime.date(2023, 11, 11)

In [19]:
if date_today > last_day_data:
    # The target day for data update
    target_day = date_today if now.time() > datetime.time(15, 00) else date_today + datetime.timedelta(days=-1)
    start_day = last_day_data #+ datetime.timedelta(days=1)
    end_day = target_day + datetime.timedelta(days=1)

    if start_day <= target_day:
       
        update_data_df = yf.download(['2330.TW'], start=start_day.strftime('%Y-%m-%d'), end=end_day.strftime('%Y-%m-%d'), progress=False)  # For test run

        # Update the data df and save to file
        if len(update_data_df.index) > 0:  # Double-check new data is available
            symbols = stock_symbol_df['yf_symbol'].to_list()
            update_data_df = yf.download(symbols, start=start_day.strftime('%Y-%m-%d'), end=end_day.strftime('%Y-%m-%d'), progress=True)
            data_df = pd.concat([data_df, update_data_df])
            data_df.to_feather(data_filepath)
        else:
            print('New data is not available. Maybe try later.')
    else:
        print('No update')

print(f'Last three days of the data: {data_df.tail(3).index.to_list()}')

Last three days of the data: [Timestamp('2023-11-08 00:00:00'), Timestamp('2023-11-09 00:00:00'), Timestamp('2023-11-10 00:00:00')]


In [20]:
# Remove duplicate dates just in case
data_df = data_df.reset_index().drop_duplicates(subset=[('Date', '')], keep='first').set_index('Date')
data_df.to_feather(data_filepath)